# Data pre-processing

## Setup -CHECK SPELLING MISTAKES

In [1]:
# Install a pip package in the current Jupyter kernel - REMOVE IN PROD
#import sys
#!{sys.executable} -m pip install gtfparse

Import relevant libraries

In [2]:
# Import libs
import os.path
from os import path
import gzip
import pandas as pd
import numpy as np
import urllib.request
from gtfparse import read_gtf
from biomart import BiomartServer

Create function for printing out rows and cols

In [3]:
def print_rowcol(pretext, df):
    template = pretext + ': {} rows x {} cols'
    print(template.format(str(df.shape[0]), str(df.shape[1])))

Setup paths - main data path is folder inside the code repository where the data dowloaded from download-data has been stored

In [4]:
data_path = '../data'
gtf_path = data_path + '/Mus_musculus.GRCm38.99.gtf'
tpm_yang_path = data_path + '/GSE90848_Ana6_basal_hair_bulb_TPM.txt'
tpm_yang_path2 = data_path + '/GSE90848_Tel_Ana1_Ana2_bulge_HG_basal_HB_TPM.txt'
tpm_joost_path = data_path + '/GSE67602_Joost_et_al_expression.txt'
tpm_ghahramani_path = data_path + '/GSE99989_NCA_BCatenin_TPM_matrix.csv'
gene_tsv_path = data_path + '/gene_names.tsv'
gtf_data_path = data_path + '/gtf.txt'

Set the main thresholds for the pre-processing
- min_num_genes_in_cell_exp - is the min number of genes that need to have expression > **min_ltpm_exp** for it to be considered a valid cell
- min_num_cells_for_gene_exp - is the min number of cells that need to have an expression for that gene > **min_ltpm_exp** for it to be considered a valid gene

In [5]:
# Thresholds
min_num_genes_in_cell_exp = 1000
min_num_cells_for_gene_exp = 10
min_ltpm_exp = 1

## Load and process gene data

Load GTF data and show some data

In [6]:
if path.exists(gtf_data_path) is False:
    df_gtf = read_gtf(gtf_path)
    df_gtf.to_csv(gtf_data_path)

df_gtf = pd.read_csv(gtf_data_path)
print_rowcol('Loaded GTF', df_gtf)
#df_gtf

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (1,24) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Loaded GTF: 1868330 rows x 27 cols


Calculate feature lengths

In [7]:
df_gtf.insert(6,"feature_len", df_gtf.end - df_gtf.start)
#df_gtf

INFO:numexpr.utils:NumExpr defaulting to 4 threads.


Filter GTF data for exons, 3' and 5' UTRs

In [8]:
df_gtf = df_gtf[(df_gtf.feature=='exon') | (df_gtf.feature=='three_prime_utr') | (df_gtf.feature=='five_prime_utr')]
print_rowcol('Filtered GTF for exons and UTR', df_gtf)
#df_gtf

Filtered GTF for exons and UTR: 1025718 rows x 28 cols


Aggregate the feature lengths to find a transcript length for each gene

In [9]:
df_gtf_transcript_len = pd.DataFrame(df_gtf.groupby(['gene_id']).sum()['feature_len'])
#df_gtf_transcript_len

Check it makes sense

In [10]:
#df_gtf_transcript_len[df_gtf_transcript_len.index.isin(['ENSMUSG00000051951'])]

In [11]:
#df_gtf[df_gtf.gene_id.isin(['ENSMUSG00000051951'])]

In [12]:
del df_gtf

## Resolve Gene names

Load gene names

In [13]:
df_gene_names = pd.read_csv(gene_tsv_path, sep='\t', header=None)
df_gene_names.columns = ['gene_id', "gene_name"]
df_gene_names.index = df_gene_names.gene_id
df_gene_names = df_gene_names.drop('gene_id', axis=1)
print_rowcol('Loaded gene names', df_gene_names)
#df_gene_names

Loaded gene names: 56289 rows x 1 cols


In [14]:
df_gene_name2id = pd.read_csv(gene_tsv_path, sep='\t', header=None)
df_gene_name2id.columns = ['gene_id', "gene_name"]
df_gene_name2id.index = df_gene_name2id.gene_name
df_gene_name2id = df_gene_name2id.drop('gene_name', axis=1)
print_rowcol('Loaded gene names', df_gene_name2id)
#df_gene_name2id

Loaded gene names: 56289 rows x 1 cols


## Load in RNA-Seq data

### Yang

Read the yang1 data set and look at the data

In [15]:
df_tpm_1 = pd.read_csv(tpm_yang_path, sep='\t')
print_rowcol('Loaded Yang1', df_tpm_1)
#df_tpm_1

Loaded Yang1: 46609 rows x 385 cols


Clean the yang1 dataset to get the gene names, call `.shape` to check we havent filtered anything

In [16]:
# Split gene ids on _ and load into a new data frame and set the columns
split_data = pd.DataFrame(df_tpm_1.Gene_id.str.split("_", expand=True))
split_data.columns = ["Gene_id", "Gene_name", "Gene_name2"]

# Fill in the NA's with blank strings
split_data["Gene_name"] = split_data.Gene_name.fillna('')
split_data["Gene_name2"] = split_data.Gene_name2.fillna('')

# Concatinate the strings that have split more than once back to their standard for e.g GENEID_GENENAME_SOMEMORENAME
split_data["Gene_name"] = split_data.apply(lambda x: x.Gene_name if x.Gene_name2 == '' else x.Gene_name + '_' + x.Gene_name2, axis=1)
print_rowcol('Filter check', split_data)

Filter check: 46609 rows x 3 cols


Write the gene names back into the main dataset, print out dataset to check we do indeed have gene names where they were available

In [17]:
# Insert the columns back into the main data array
df_tpm_1["Gene_id"] = split_data.Gene_id
df_tpm_1.insert(1,"Gene_name", split_data.Gene_name)
#df_tpm_1

Set gene name to index after checking it is unique

In [18]:
df_tpm_1.Gene_id.is_unique

True

In [19]:
df_tpm_1.index = df_tpm_1['Gene_id']
#df_tpm_1

Load in Yang2 

In [20]:
df_tpm_2 = pd.read_csv(tpm_yang_path2, sep='\t')
print_rowcol('Loaded Yang2', df_tpm_2)
#df_tpm_2

Loaded Yang2: 46609 rows x 399 cols


In [21]:
# Split gene ids on _ and load into a new data frame and set the columns
split_data = pd.DataFrame(df_tpm_2.Gene_id.str.split("_", expand=True))
split_data.columns = ["Gene_id", "Gene_name", "Gene_name2"]

# Fill in the NA's with blank strings
split_data["Gene_name"] = split_data.Gene_name.fillna('')
split_data["Gene_name2"] = split_data.Gene_name2.fillna('')

# Concatinate the strings that have split more than once back to their standard for e.g GENEID_GENENAME_SOMEMORENAME
split_data["Gene_name"] = split_data.apply(lambda x: x.Gene_name if x.Gene_name2 == '' else x.Gene_name + '_' + x.Gene_name2, axis=1)
print_rowcol('Filter check', split_data)

Filter check: 46609 rows x 3 cols


In [22]:
# Insert the columns back into the main data array
df_tpm_2["Gene_id"] = split_data.Gene_id
df_tpm_2.insert(1,"Gene_name", split_data.Gene_name)
#df_tpm_2

Set gene name to index after checking it is unique

In [23]:
df_tpm_2.Gene_id.is_unique

True

In [24]:
df_tpm_2.index = df_tpm_2['Gene_id']
#df_tpm_2

## Joost / Kasper

Load in data

In [25]:
df_rpk = pd.read_csv(tpm_joost_path, sep='\t')
print_rowcol('Loaded Kasper', df_rpk)
#df_rpk

Loaded Kasper: 26024 rows x 1423 cols


### Convert from FPKM (Fragments Per Kilobase Million) to TPM (Transcripts Per Kilobase Million)

Rename gene/cell column

In [26]:
df_rpk.rename( columns={'Gene\Cell':'gene_name'}, inplace=True)
df_rpk.head()

,gene_name,1772067055_A01,1772067055_A03,1772067055_A04,1772067055_A05,1772067055_A06,1772067055_A07,1772067055_A09,1772067055_B01,1772067055_B02,...,1772072285_C12,1772072285_D01,1772072285_D03,1772072285_D05,1772072285_D06,1772072285_D09,1772072285_D12,1772072285_E02,1772072285_E06,1772072285_G06
0,ERCC-00002,363,330,267,262,334,328,280,318,297,...,945,850,696,544,701,817,601,687,589,229
1,ERCC-00003,20,17,14,16,20,26,14,8,25,...,82,79,56,36,54,92,30,64,47,32
2,ERCC-00004,276,258,215,190,244,259,209,235,237,...,1000,925,755,667,745,1051,609,749,534,287
3,ERCC-00009,25,14,14,7,20,31,20,26,24,...,56,49,42,27,33,49,30,44,27,13
4,ERCC-00012,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Set gene to index after checking unique

In [27]:
df_rpk.index.is_unique

True

In [28]:
df_rpk.index = df_rpk['gene_name']
#df_rpk

Insert gene_id into data

In [29]:
df_rpk_merged = df_rpk
df_rpk_merged = df_rpk_merged.drop('gene_name', axis=1)
df_rpk_merged = df_gene_name2id.join(df_rpk_merged, lsuffix='', rsuffix='', how='inner')
df_rpk_merged.index = df_rpk_merged.gene_id
df_rpk_merged = df_rpk_merged.drop('gene_id', axis=1)
print_rowcol('Shape after merging gene ids', df_rpk_merged)
#df_rpk_merged

Shape after merging gene ids: 21827 rows x 1422 cols


Get length of each gene into data frame

In [30]:
df_rpk_merged_len = df_gtf_transcript_len.join(df_rpk_merged, lsuffix='', rsuffix='', how='inner')
print_rowcol('Shape after merging gene feature_len', df_rpk_merged_len)
#df_rpk_merged_len

Shape after merging gene feature_len: 21522 rows x 1423 cols


Sum the read counts per sample

In [31]:
df_scaling_factor = pd.DataFrame(df_rpk_merged_len.sum(axis=0) / 1000000)
df_scaling_factor.columns = ['scaling_factor']
df_scaling_factor = df_scaling_factor.drop('feature_len')

Divide the read counts by the length of each gene in kilobases. This gives you reads per kilobase (RPK)

In [32]:
df_rpk_merged_len = df_rpk_merged_len.iloc[:,1:].div(df_rpk_merged_len.feature_len, axis=0)

Divide the RPK values by the “per million” scaling factor. This gives you TPM.

In [ ]:
df_rpk_merged_len = df_rpk_merged_len.iloc[:,:].div(df_scaling_factor.T, axis=0)
df_rpk_merged_len

In [ ]:



#df_rpk_merged_len = df_rpk_merged_len / df_rpk_merged_len.feature_len
#df_rpk_merged_len

In [ ]:
fkjfkjfkjfkjf q=49=2orq\fk]3gi1-t

In [ ]:
df_rpk_merged_len

Calculate the 'per million' scaling factor

## Ghahramani

In [ ]:
Rename gene_name column

In [ ]:
df_rpk.rename( columns={'Unnamed: 0':'gene_name'}, inplace=True )
df_rpk.head()

## Create merged dataset

Create merged dataset from all subsets

In [ ]:
df_tpm_combined = df_tpm_1.join(df_tpm_2, lsuffix='', rsuffix='_other', how='inner')
df_tpm_combined.drop(['Gene_id_other', 'Gene_name_other'], axis=1)
df_tpm_combined.shape

Create data only set (this is easier for tensorflow to deal with)

In [ ]:
df_tpm_combined_dataonly = df_tpm_combined.drop(['Gene_id', 'Gene_name'], axis=1)
df_tpm_combined_dataonly = df_tpm_combined_dataonly.select_dtypes(exclude=['object'])
df_tpm_combined_dataonly.shape

Create `log2(TPM+1)` dataset

In [ ]:
df_ltpm_combined_dataonly = np.log2(df_tpm_combined_dataonly + 1)
df_ltpm_combined_dataonly

Filter for genes and cells which match filtering criteria. Call `.shape` to check what was filtered

In [ ]:
df_expression_filt_mask = df_ltpm_combined_dataonly > min_ltpm_exp
df_ltpm_combined_dataonly_genefilt = df_ltpm_combined_dataonly[df_expression_filt_mask.sum(axis=1) > min_num_cells_for_gene_exp]
df_ltpm_combined_dataonly_genefilt.shape

In [ ]:
df_ltpm_combined_dataonly_genecellfilt = df_ltpm_combined_dataonly_genefilt \
    .T[(df_ltpm_combined_dataonly_genefilt > min_ltpm_exp).sum(axis=0) > min_num_genes_in_cell_exp]
df_ltpm_combined_dataonly_genecellfilt = df_ltpm_combined_dataonly_genecellfilt.T
df_ltpm_combined_dataonly_genecellfilt.shape

## Write data to file

Get the column and row names as a list

In [ ]:
df_column_names = pd.DataFrame(list(df_ltpm_combined_dataonly_genecellfilt.columns.values))
df_row_names = pd.DataFrame(list(df_ltpm_combined_dataonly_genecellfilt.index.values))

print(df_column_names.shape)
print(df_row_names.shape)

Write the data to file

In [ ]:
df_ltpm_combined_dataonly_genecellfilt.to_csv(data_path + '/tpm_combined.csv', index=False, header=False)
df_column_names.to_csv(data_path + '/tpm_combined_cols.csv', index=False, header=False)
df_row_names.to_csv(data_path + '/tpm_combined_rows.csv', index=False, header=False)